In [1]:
import torch

try:
    import google.colab
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# Make use of a GPU or MPS (Apple) if one is available.  (see module 3.2)
import torch
has_mps = torch.backends.mps.is_built()
device = "mps" if has_mps else "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Note: not using Google CoLab
Using device: cpu


In [3]:
import pandas as pd
import urllib.request
import os

# Set Pandas display options
pd.set_option('display.max_columns', 6)
pd.set_option('display.max_rows', 5)

# Download the file using urllib
url = 'https://github.com/jeffheaton/jheaton-ds2/raw/main/kdd-with-columns.csv'
filename = 'DMZServiciosN4_Web_U_EX240225.log'

if not os.path.isfile(filename):
    try:
        urllib.request.urlretrieve(url, filename)
    except:
        print('Error downloading')
        raise

print(filename)
# Cargo titulos de columnas
with open(filename, 'r') as file:
    for line in file:
        if line.startswith('#Fields:'):
            # Eliminar el prefijo y el salto de línea, luego dividir por espacio
            columnanombre = line.replace('#Fields: ', '').strip().split()
            break

print(columnanombre)


# Se cargan los datos con las columnas a utilizar
cols_to_use = ['time', 'cs-method', 'cs-uri-stem', 'cs-username','c-ip','cs-host','sc-status'
                                                         ,'sc-bytes','cs-bytes','time-taken']

df = pd.read_csv(filename, sep=' ', names=columnanombre, comment='#',usecols=cols_to_use)
print("Read {} rows.".format(len(df)))

# Display 5 rows
pd.set_option('display.max_columns', 15)
pd.set_option('display.max_rows', 5)
              
print(df)

DMZServiciosN4_Web_U_EX240225.log
['date', 'time', 's-computername', 's-ip', 'cs-method', 'cs-uri-stem', 'cs-uri-query', 's-port', 'cs-username', 'c-ip', 'cs(User-Agent)', 'cs(Referer)', 'cs-host', 'sc-status', 'sc-substatus', 'sc-win32-status', 'sc-bytes', 'cs-bytes', 'time-taken']
Read 232076 rows.
            time cs-method                cs-uri-stem cs-username  \
0       00:00:00      POST       /LuciaWS/awsdua.aspx        0642   
1       00:00:00       GET  /luciaws/axmovstkxml.aspx     TRN0038   
...          ...       ...                        ...         ...   
232074  23:59:58      POST       /LuciaWS/awsdua.aspx        0642   
232075  23:59:59      POST  /LuciaWsPr/awsprcrpt.aspx     OTR4085   

                  c-ip                   cs-host  sc-status  sc-bytes  \
0        100.42.52.211  servicios.aduanas.gub.uy        200      3397   
1       200.58.138.180  servicios.aduanas.gub.uy        200      1813   
...                ...                       ...        ...     

In [ ]:
### Comienza el trabajo con los datos para CATEGORIZAR LOS CAMPOS

In [4]:
### Campo sc-status 
# elimanmos todos los errores del aplicativo y solo nos quedamos con los 200 (ok)
#   Al eliminar esta columna evitamos tener aplitivos que no existen o sin usuario (401 y 404) ademas de erres del servidor 500.
df = df[df['sc-status'] == 200]

## eliminamos la columna sc-status
df = df.drop('sc-status', axis=1)


In [5]:

### Campo cs-uri-stem o campo del aplicativo a invocar


# PARA NORMALIZAR - Reemplazar 'dna\' o 'DNA\' al comienzo de 'cs-username' por una cadena vacía, insensible a mayúsculas/minúsculas 
df['cs-uri-stem'] = df['cs-uri-stem'].str.split('/').str[-1]
                                                    

# Agrupar , contar, ordenar y obtener los 19 mas frecuentes  y el 20 va a ser el resto 'otros'
top_20_csuristem = df.groupby('cs-uri-stem')['cs-uri-stem'].count().sort_values(ascending=False).head(19).index.tolist()


# Crear una nueva columna que categoriza 'cs-username'
df['cs-uri-stem-cat'] = df['cs-uri-stem'].apply(lambda x: x if x in top_20_csuristem else 'otros')

# Convertir la nueva columna categorizada a variables one-hot-encoding (vector de 20  bits donde solo esta activo el que corresponde) 
csuristem_dummies = pd.get_dummies(df['cs-uri-stem-cat'])

print(csuristem_dummies)

# Guardar la lista en un archivo de texto  las categorias que seran necesarias para evaluar el trafico neuvo con la misma categorizacion

with open('lista_categorias_csuristem.csv', 'w') as archivo:
    for elemento in top_20_csuristem:
        archivo.write("%s\n" % elemento)



        aWSPrcRpt.aspx  aWSPrcRptV2.aspx  aavisoretiropreembarquead.aspx  \
0                False             False                           False   
1                False             False                           False   
...                ...               ...                             ...   
232074           False             False                           False   
232075           False             False                           False   

        afxmldua.aspx  afxmlhis.aspx  aingresoaterminalad.aspx  \
0               False          False                     False   
1               False          False                     False   
...               ...            ...                       ...   
232074          False          False                     False   
232075          False          False                     False   

        atradwsds.aspx  ...  awsdua.aspx  awsestsrvprcele.aspx  \
0                False  ...         True                 False   
1             

In [6]:

### Campo time
## Realizaremos una particion del tieme en 4 categorias . de 23 a 6 (hora de alto hacking), 6 a 9 y 19 a 23 y el central de 9 a 19
# Para ello se parte de un vector binario one-hot-encoding de 3 elementos (23 a 6 = 100 , de 9 a 19 = 010 y resto = 001

# Convertir la columna 'time' a datetime
# Dividir la cadena por ':' y quedarse con la parte antes del primer ':'
def categorizar_por_hora(time):
    # Primero, dividimos el string para extraer la hora
    hora = int(time.split(':')[0])  # Convertimos a entero el primer elemento luego de hacer split por ':'
    
    # Ahora, categorizamos según los rangos definidos
    if (6 <= hora < 9) or (19 <= hora < 23):
        return '6a9y19a23'
    elif 9 <= hora < 19:
        return '9a19'
    elif hora >= 23 or hora < 6:
        return '23a6'
    else:
        return 'Otro' 

# Aplicar la función para categorizar
df['time-cat'] = df['time'].apply(categorizar_por_hora)

              

# Agrupar , contar, ordenar y obtener los 19 mas frecuentes  y el 20 va a ser el resto 'otros'
lista_timeCategoria = ['6a9y19a23','9a19','23a6']



# Convertir la nueva columna categorizada a variables one-hot-encoding (vector de 20  bits donde solo esta activo el que corresponde) 
time_dummies = pd.get_dummies(df['time-cat'])

print(time_dummies)

# Guardar la lista en un archivo de texto  las categorias que seran necesarias para evaluar el trafico neuvo con la misma categorizacion

with open('lista_categorias_time.csv', 'w') as archivo:
    for elemento in lista_timeCategoria:
        archivo.write("%s\n" % elemento)



        23a6  6a9y19a23   9a19
0       True      False  False
1       True      False  False
...      ...        ...    ...
232074  True      False  False
232075  True      False  False

[170026 rows x 3 columns]


In [7]:

### Campo cs-username


# PARA NORMALIZAR - Reemplazar 'dna\' o 'DNA\' al comienzo de 'cs-username' por una cadena vacía, insensible a mayúsculas/minúsculas 
df['cs-username'] = df['cs-username'].str.replace('^(?i)dna\\\\', '', regex=True)

# Agrupar por 'cs-username', contar, ordenar y obtener los 19 mas frecuentes  y el 20 va a ser el resto 'otros'
top_20_usernames = df.groupby('cs-username')['cs-username'].count().sort_values(ascending=False).head(19).index.tolist()


# Crear una nueva columna que categoriza 'cs-username'
df['cs-username-cat'] = df['cs-username'].apply(lambda x: x if x in top_20_usernames else 'otros')

# Convertir la nueva columna categorizada a variables one-hot-encoding (vector de 20  bits donde solo esta activo el que corresponde al usuario) 
username_dummies = pd.get_dummies(df['cs-username-cat'])

print(username_dummies)

# Guardar la lista en un archivo de texto  las categorias que seran necesarias para evaluar el trafico neuvo con la misma categorizacion

with open('lista_categorias_csusername.csv', 'w') as archivo:
    for elemento in top_20_usernames:
        archivo.write("%s\n" % elemento)



         0063   0642   3001   3004   6084  ANPWS  DNMDNA  ...  bullmon  \
0       False   True  False  False  False  False   False  ...    False   
1       False  False  False  False  False  False   False  ...    False   
...       ...    ...    ...    ...    ...    ...     ...  ...      ...   
232074  False   True  False  False  False  False   False  ...    False   
232075  False  False  False  False  False  False   False  ...    False   

        otr0624  otr0893  otr5690  otr9981  otros  trn0038  
0         False    False    False    False  False    False  
1         False    False    False    False   True    False  
...         ...      ...      ...      ...    ...      ...  
232074    False    False    False    False  False    False  
232075    False    False    False    False  False    False  

[170026 rows x 20 columns]


In [8]:
### Campo cs-method



# PARA NORMALIZAR - crearemos un vector de 3 bit dodne (100 = GET; 010 = Put ; 001 = Otro
categoria_csmethod = ['GET','POST','OTROS']

# Crear una nueva columna que categoriza 'cs-username'
df['cs-method-cat'] = df['cs-method'].apply(lambda x: x if x in categoria_csmethod else 'otros')

# Convertir la nueva columna categorizada a variables one-hot-encoding (vector de 20  bits donde solo esta activo el que corresponde al usuario) 
method_dummies = pd.get_dummies(df['cs-method-cat'])

print(method_dummies)

# Guardar la lista en un archivo de texto  las categorias que seran necesarias para evaluar el trafico neuvo con la misma categorizacion

with open('lista_categorias_csmethod.csv', 'w') as archivo:
    for elemento in categoria_csmethod:
        archivo.write("%s\n" % elemento)

          GET   POST  otros
0       False   True  False
1        True  False  False
...       ...    ...    ...
232074  False   True  False
232075  False   True  False

[170026 rows x 3 columns]


In [9]:
### Campo cs-ip y vencindad

import pandas as pd
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

# Suponiendo que df es tu DataFrame con una columna 'ip_address'
df['ip_split'] = df['c-ip'].str.split('.')
df[['octet1', 'octet2', 'octet3', 'octet4']] = pd.DataFrame(df['ip_split'].tolist(), index=df.index).astype(int)

# Ahora df tiene cuatro columnas nuevas: 'octet1', 'octet2', 'octet3', 'octet4'

# Convertir las columnas de octetos a una matriz numérica y escalar los datos
X = StandardScaler().fit_transform(df[['octet1', 'octet2', 'octet3', 'octet4']])

# Aplicar DBSCAN
# eps es la distancia máxima entre dos muestras para que una sea considerada en la vecindad de la otra
# min_samples es el número de muestras en una vecindad para que un punto sea considerado un punto central
db = DBSCAN(eps=0.5, min_samples=5).fit(X)

# Las etiquetas de los clusters
labels = db.labels_

# Agregar las etiquetas al DataFrame original
df['cluster'] = labels

# Cantidad de clusters en las etiquetas, ignorando el ruido si es presente
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)

print(f'Número estimado de clusters: {n_clusters_}')
print(df)




Número estimado de clusters: 51
            time cs-method       cs-uri-stem cs-username            c-ip  \
0       00:00:00      POST       awsdua.aspx        0642   100.42.52.211   
1       00:00:00       GET  axmovstkxml.aspx     TRN0038  200.58.138.180   
...          ...       ...               ...         ...             ...   
232074  23:59:58      POST       awsdua.aspx        0642   100.42.52.211   
232075  23:59:59      POST    awsprcrpt.aspx     OTR4085    200.40.42.95   

                         cs-host  sc-bytes  ...  cs-method-cat  \
0       servicios.aduanas.gub.uy      3397  ...           POST   
1       servicios.aduanas.gub.uy      1813  ...            GET   
...                          ...       ...  ...            ...   
232074  servicios.aduanas.gub.uy      3253  ...           POST   
232075  servicios.aduanas.gub.uy      1093  ...           POST   

                   ip_split octet1 octet2 octet3 octet4 cluster  
0        [100, 42, 52, 211]    100     42     52

In [ ]:
import pandas as pd

def encode_numeric_zscore(df, name):
    """
    Apply z-score normalization to a specified numeric column.

    Parameters:
    df (DataFrame): The pandas DataFrame containing the column.
    name (str): The name of the column to normalize.
    """
    mean = df[name].mean()
    sd = df[name].std()
    df[name] = (df[name] - mean) / sd

def encode_text_dummy(df, name):
    """
    Convert a categorical column to dummy variables.

    Parameters:
    df (DataFrame): The pandas DataFrame containing the column.
    name (str): The name of the categorical column.
    """
    dummies = pd.get_dummies(df[name], prefix=name, dtype=float)
    df = pd.concat([df, dummies], axis=1)
    df.drop(name, axis=1, inplace=True)
    return df

def process_dataframe(df):
    """
    Process a DataFrame by encoding its features.

    Parameters:
    df (DataFrame): The pandas DataFrame to process.
    """
    for name in df.columns:
        if name == 'checkpoint.rule_action':
            continue
        #elif df[name].dtype == bool:
        #    print("**", name)
        #    df[name] = df[name].astype(float)
    # """    elif name in ['destination.port', 'source.port'
        # , '@timestamp', 'logged_in',, 'service'
             #         'source.ip', 'destination.ip'
            #          ]:      encode_numeric_zscore(df, name) """
        else:
            df = encode_text_dummy(df, name)
    return df

In [ ]:
""" df.drop(df.columns[0], axis=1, inplace=True) """
print(df)
print(df.head())

                     event.created        source.ip source.port  \
17747  Feb 21, 2024 @ 21:20:06.263  192.168.250.216      22,584   
8445   Feb 21, 2024 @ 21:24:04.419   200.40.211.251      50,859   
...                            ...              ...         ...   
3441   Feb 21, 2024 @ 21:26:15.271    179.27.47.201      29,628   
18653  Feb 21, 2024 @ 21:19:41.420    179.27.96.252      58,266   

        destination.ip destination.port source.geo.country_name  \
17747          8.8.8.8               53                       -   
8445   192.168.245.254           18,234                 Uruguay   
...                ...              ...                     ...   
3441    200.40.211.236              443                 Uruguay   
18653   200.40.211.236              443                 Uruguay   

                         source.as.organization.name  \
17747                                              -   
8445   Administracion Nacional de Telecomunicaciones   
...                       

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16328 entries, 2247 to 12644
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   event.created                 16328 non-null  object
 1   source.ip                     16328 non-null  object
 2   source.port                   16328 non-null  object
 3   destination.ip                16328 non-null  object
 4   destination.port              16328 non-null  object
 5   source.geo.country_name       16328 non-null  object
 6   source.as.organization.name   16328 non-null  object
 7   destination.geo.country_name  16328 non-null  object
 8   network.application           16328 non-null  object
 9   destination.nat.ip            16328 non-null  object
 10  event.type                    16328 non-null  object
 11  network.transport             16328 non-null  object
 12  checkpoint.rule_action        16328 non-null  object
 13  network.direc

In [ ]:
for col in df.columns:
    try:
        df[col].astype(float)
    except ValueError:
        print(f"Column '{col}' contains non-numeric values.")

Column 'event.created' contains non-numeric values.
Column 'source.ip' contains non-numeric values.
Column 'source.port' contains non-numeric values.
Column 'destination.ip' contains non-numeric values.
Column 'destination.port' contains non-numeric values.
Column 'source.geo.country_name' contains non-numeric values.
Column 'source.as.organization.name' contains non-numeric values.
Column 'destination.geo.country_name' contains non-numeric values.
Column 'network.application' contains non-numeric values.
Column 'destination.nat.ip' contains non-numeric values.
Column 'event.type' contains non-numeric values.
Column 'network.transport' contains non-numeric values.
Column 'checkpoint.rule_action' contains non-numeric values.
Column 'network.direction' contains non-numeric values.


In [ ]:
non_numeric_column = "checkpoint.rule_action"  # Replace with the actual column name
print(df[non_numeric_column].head())

17747    Accept
8445     Accept
20364      Drop
18416    Accept
9619       Drop
Name: checkpoint.rule_action, dtype: object


In [ ]:
pd.set_option('display.max_columns', 6)
pd.set_option('display.max_rows', 5)

df = process_dataframe(df)
df.dropna(inplace=True, axis=1)
print(df.head())

In [ ]:
normal_mask = df['checkpoint.rule_action']=='Accept'
attack_mask = df['checkpoint.rule_action']!='Drop'

df.drop('checkpoint.rule_action',axis=1,inplace=True)

df_normal = df[normal_mask]
df_attack = df[attack_mask]
df_normal = process_dataframe(df_attack)
df_normal.dropna(inplace=True, axis=1)
df_attack.dropna(inplace=True, axis=1)
df_attack = process_dataframe(df_normal)


print(df.head())

print(f"Normal count: {len(df_normal)}")
print(f"Attack count: {len(df_attack)}")

<ipython-input-9-13901c134c89>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_attack.dropna(inplace=True, axis=1)


In [ ]:
# This is the numeric feature vector, as it goes to the neural net
x_normal = df_normal.values
x_attack = df_attack.values

In [ ]:
from sklearn.model_selection import train_test_split

x_normal_train, x_normal_test = train_test_split(
    x_normal, test_size=0.25, random_state=42)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Convert numpy arrays to PyTorch tensors and move them to the appropriate device
x_normal_train_tensor = torch.tensor(x_normal_train).float().to(device)
x_normal_tensor = torch.tensor(x_normal).float().to(device)
x_attack_tensor = torch.tensor(x_attack).float().to(device)

# Create DataLoader for batch processing
train_data = TensorDataset(x_normal_train_tensor, x_normal_train_tensor)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)

# Define the model using Sequential
model = nn.Sequential(
    nn.Linear(x_normal.shape[1], 25),
    nn.ReLU(),
    nn.Linear(25, 3),
    nn.ReLU(),
    nn.Linear(3, 25),
    nn.ReLU(),
    nn.Linear(25, x_normal.shape[1])
).to(device)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
# Training loop
for epoch in range(num_epochs):
    running_loss = 0
    den = 0
    for data in train_loader:
        inputs, targets = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss +=loss.item()
        den+=1

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss/den}')
    running_loss = 0.0

Epoch [1/10], Loss: 0.015089166516970311
Epoch [2/10], Loss: 0.0017625625727565161
Epoch [3/10], Loss: 0.0011732291958261548
Epoch [4/10], Loss: 0.001122224926283317
Epoch [5/10], Loss: 0.0011104200322214247
Epoch [6/10], Loss: 0.0011081521203907738
Epoch [7/10], Loss: 0.0011082047442739298
Epoch [8/10], Loss: 0.0011075983680452087
Epoch [9/10], Loss: 0.0011081941400854184
Epoch [10/10], Loss: 0.0011082928270168071


In [ ]:
print(f"Normal train count: {len(x_normal_train)}")
print(f"Normal test count: {len(x_normal_test)}")

Normal train count: 1779
Normal test count: 593


In [ ]:
model.eval()  # Set the model to evaluation mode

# Function to calculate RMSE
def calculate_rmse(model, data):
    with torch.no_grad():
        predictions = model(data)
        mse_loss = nn.MSELoss()(predictions, data)
    return torch.sqrt(mse_loss).item()

# Evaluating the model
score1 = calculate_rmse(model, torch.tensor(x_normal_test).float().to(device))
score2 = calculate_rmse(model, x_normal_tensor)
score3 = calculate_rmse(model, x_attack_tensor)

print(f"Out of Sample Normal Score (RMSE): {score1}")
print(f"Insample Normal Score (RMSE): {score2}")
print(f"Attack Underway Score (RMSE): {score3}")

Out of Sample Normal Score (RMSE): 0.033687397837638855
Insample Normal Score (RMSE): 0.033377647399902344
Attack Underway Score (RMSE): 0.03433653712272644
